# Lab 2: MD Simulations

In this lab we will run MD simulations in simple 1D and 2D model systems to understand the basic principles.

First we will load several modules from the ensembler library

In [ ]:
import tempfile
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

#Ensembler
##COde
from ensembler.potentials import OneD as potentials1D
from ensembler.potentials import TwoD as potentials2D

from ensembler.samplers.stochastic import langevinIntegrator, langevinVelocityIntegrator
from ensembler.samplers.optimizers import conjugate_gradient
from ensembler.conditions.box_conditions import periodicBoundaryCondition

from ensembler.system import system

##Visualisation
import ensembler.visualisation.plotPotentials as vis
from ensembler.visualisation.plotPotentials import plot_1DPotential
from ensembler.visualisation.plotSimulations import simulation_analysis_plot
from ensembler.visualisation.animationSimulation import animation_trajectory

number_of_points = 1000

## Potential definition
We will use a four-well potential describing four local minima (=states/conformations) along a 1D (reaction) coordinate 

In [ ]:
# accessible space
space_range = [0, 10]
positions = np.linspace(space_range[0], space_range[1], number_of_points)

# build potential
V = potentials1D.fourWellPotential(a=1, b=3.5, c=6, d=8)

fig, axes = vis.plot_potential(V, positions=positions)
# fig.savefig("four_well.png")

# Energy minimization using Conjugate Gradient method
A frequent task in molecular mechanics is energy minimization or geometry optimization. 

Define number of minimization steps and initial position

In [ ]:
sim_steps = 25
init_position = 10

In [ ]:
# settings
sampler = conjugate_gradient(max_step_size=0.5)
sys = system(potential=V, sampler=sampler, start_position=init_position)

# simulate
cur_state = sys.simulate(sim_steps, withdraw_traj=True)
print()
print("last_state: ", cur_state)

In the visualization we see that the particle quickly leaves its starting position and moves energetically downhill. The particle finally resides at the next local energy minimum. Note, that this minimization algorithm only finds the nearest local minimum. It does not assure that the global optimum is found.

In [ ]:
# animation
ani, out_path = animation_trajectory(
    sys, limits_coordinate_space=space_range, title="Conjugated Gradient Simulation")
ani

In [ ]:
# plot
simulation_analysis_plot(sys, title="Conjugated Gradient",
                         limits_coordinate_space=space_range)

# Homework, Problem 1

Please write a code that uses only conjugate gradient minimizations to automatically ...
a) ... identify all local minima
b) ... identifies the global minimum among the local minima.

Please make snapshot of code and paste it into document for submission. Also add position and potential energy values for all local minima.

Note: You can perform energy minimization multiple times from different starting points.

# End Homework, Problem 1

_______________________________________________

# MD Simulation

We will perform MD simulations using the Langevin integrator to model the particle motion in a solvent. The Langevin integrator assumes that the system is coupled to a heat-bath. The interaction between the system and the heat-bath is modeled by a stochastic force term. 

The user has to set the step size $dt$ and the friction coefficient $gamma$. The initial velocity will be randomly drawn from a Maxwell-Boltzmann distribution. The temperature of the simulation and starting position will be set during the system setup (see below).

In [ ]:
# Simple Langevin integration simulation:
# Thermostat is already included (Langevin thermostat)
sim_steps = 500
time_step = 0.2
start_position = 8
temperature = 0.6
space_range = [0, 10]

# integrator
sampler = langevinVelocityIntegrator(
    dt=time_step, gamma=25, old_position=start_position)

We are now ready to perform the simulations. The system class wraps the integrator and the potential. Additionally, the initial position of the particle $position$ as well as the temperature parameter $temperature$ are set.

To start the simulation we define the number of steps and run `sys.simulate()`. The progress of the simulation is displayed by a progress bar.

In [ ]:
# Simulation Setup
sys = system(potential=V, sampler=sampler,
             start_position=start_position,  temperature=temperature)

# simulate
cur_state = sys.simulate(sim_steps, withdraw_traj=True, init_system=False)

After running the simulation, the simulation data can be displayed as a table using `sys.trajectory`. 

In [ ]:
print("Trajectory length: ", len(sys.trajectory))
print()
print("last_state: ", cur_state)
print(len(sys.trajectory))
sys.trajectory.head(20)

In [ ]:
# visualize
simulation_analysis_plot(sys, title="Langevin Simulation",
                         limits_coordinate_space=(0, 10))

In [ ]:
# animation
ani, out_path = animation_trajectory(
    sys, limits_coordinate_space=space_range, title="Langevin Simulation")
ani

# Homework, Bonus points
Please make appropriate changes to the simulation settings to sample all four local minima. Also, make sure that the minima are qualitatively explored with a frequency reflecting the relative potential energy of the minima.
```
sim_steps =
time_step =
start_position =
temperature =
```

# End Homework, Bonus points

___________________________________

# Simulation in 2D potential

### Settings


In [ ]:
min_x = -2*np.pi
max_x = 2*np.pi

amplitude = 1

multiplicit = 1

temperature = 1.0

# Plotting & energy evaluations:
# params
extent = (min_x, max_x)
box_length = max_x - min_x

test_timing_with_points = 1000

### Defintion of potential:
### 2D cosine function between -2*pi and 2*pi in each dimension

In [ ]:
# 2D


V = potentials2D.wavePotential(amplitude=(
    amplitude, amplitude), radians=True, multiplicity=[multiplicit, multiplicit])
# print(V)

positions = np.linspace(min_x, max_x, test_timing_with_points)
x_positions, y_positions = np.meshgrid(positions, positions)
positions2D = np.array([x_positions.flatten(), y_positions.flatten()]).T

fig, axes = vis.plot_potential(V, positions2D)

### Run simulation using Langevin integrator and periodic boundary conditions

In [15]:
sampler = langevinIntegrator(dt=0.1, gamma=10)
periodic_cond = periodicBoundaryCondition(
    boundary=[[min_x, max_x], [min_x, max_x]])
sys = system(potential=V, sampler=sampler, start_position=[
             np.pi, np.pi], temperature=1, conditions=[periodic_cond])

# Simulate
sys.simulate(steps=100000)

# Visualize
# phase space to be visualized
positions = np.linspace(start=min_x, stop=max_x, num=1000)
simulation_analysis_plot(system=sys, title="Langevin Simulation",
                         limits_coordinate_space=[[min_x, max_x], [min_x, max_x]])